In [0]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.ml import Pipeline
from pyspark.ml.feature import PCA

In [0]:
dbutils.fs.cp("/FileStore/tables/song_small.zip", "/tmp/test.zip")

Out[2]: True

In [0]:
%sh
unzip /dbfs/tmp/test.zip

Archive: /dbfs/tmp/test.zip
 inflating: song_small.csv

In [0]:
dbutils.fs.mv("file:/databricks/driver/song_small.csv", "dbfs:/tmp/song_small.csv")  

Out[4]: True

In [0]:
df = spark.read.format("csv").option("inferSchema", "true").option("header","true").load("dbfs:/tmp/song_small.csv")
display(df.summary())

summary artist_familiarity artist_hotttnesss artist_id artist_location artist_mbtags artist_mbtags_count artist_name artist_terms artist_terms_freq artist_terms_weight danceability duration end_of_fade_in energy key key_confidence loudness mode mode_confidence release segments_confidence segments_loudness_max segments_loudness_max_time segments_pitches segments_timbre similar_artists song_hotttnesss song_id start_of_fade_out tempo time_signature time_signature_confidence title track_id year count 9996 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 9996 9996 mean 0.5654557053573545 0.38555220322933637 null null null null null null null null 0.0 238.7865384754741 1.039511595553203 0.13411896106644097 5.4258671688147055 0.7401509359814067 -10.447112835127719 0.8184425298027217 0.6194034022140219 19.222216268656716 43.23443718309859 38.218811369863005 21.4500325974026 34.31271865671642 12.476228461538463 30.250289868421053 0.5345081582810721 17.481605645360162 228.43919482443897 122.07944292215554 4.178519284065787 0.9539153674389347 56.60776218869622 34.50415824456881 905.4367359810974 stddev 0.16016125658075517 0.14364729904812823 null null null null null null null null 0.0 114.58346496697983 10.30163430217235 5.623729286573628 8.05771185657363 7.708022427085529 5.504553138249119 5.154539831272619 6.241908363935706 70.407884178554 124.89776791131227 85.57796570730554 67.98738069306583 82.79651628976254 55.04069698638912 78.73512701728131 6.2003264627264105 58.06249239559456 113.89924183702375 37.651181616499564 12.777642152506308 10.067554913122907 128.60924148452096 76.33548222644045 994.3059429354746 min 0.0 0.0 b'AR009211187B989185' "b""Lee's Summit Jamaica""" [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] "b""Alexander O'Neal""" "[b""rock 'n roll"" b'60s' b'50s' b'60s rock' b'00s' b'80s' b'50s lounge']" b'00s' b'60s' b'00s' b'00s' b'60s' b'00s' b'00s' b'00s' b'60s' b'00s' b'60s' b'2 step' b'60s' b'50s' b'40s' b'50s' b'00s' b'00s' b'50s' b'00s' b'00s' 25% 0.46758320106584395 0.3251485709173023 null null null null null null null null 0.0 177.26649 0.0 0.0 2.0 0.223 -13.179 0.0 0.358 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.32188268994567615 0.0 168.594 96.068 3.0 0.106 0.0 0.0 0.0 50% 0.5635835862578168 0.38072823961162366 null null null null null null null null 0.0 223.89506 0.199 0.0 5.0 0.467 -9.315 1.0 0.486 0.172 0.119 0.469 0.064 0.651 0.263 0.305 0.37929493531757236 0.44032607280862934 213.914 120.026 4.0 0.557 0.664 0.464 0.0 75% 0.6680202487233046 0.45385026533052464 null null null null null null null null 0.0 276.74077 0.427 0.0 8.0 0.658 -6.469 1.0 0.607 0.708 4.0 1.0 0.623 7.0 1.0 1.0 0.45784115609436704 1.0 266.257 143.86 4.0 0.872 89.927 5.0 2000.0 max 1.0 1.0825025567261237 b'ARZZRK91187B9A5CA5' b'san francisco' [b'zydeco', b'american', b'cajun'] [b'jazz and blues'] b'unloco' b'Pat Metheny' [b'jazz funk', b'soul jazz', b'reggae', b'roots reggae', b'rock steady', b'calypso', b'ska', b'jazz', b'future jazz', b'trip hop', b'downtempo', b'dub', b'afrobeat', b'chill-out', b'latin jazz', b'world music', b'bossa nova', b'smooth jazz', b'fusion', b'deep house', b'dancehall', b'hip hop', b'breakbeat', b'acid jazz', b'easy listening', b'disco', b'latin', b'world', b'drum and bass', b'folk', b'country', b'classic rock', b'blues', b'electronic', b'funk', b'trance', b'electro', b'indie rock', b'rock', b'electronica', b'ambient', b'house', b'guitar', b'pop', b'instrumental', b'acoustic', b'alternative', b'indie', b'soul'] [] [1.0, 1.0, 0.9305494964425336, 0.9305494964425336, 0.9305494964425336, 0.7299091663261791, 0.9305494964425336, 0.9305494964425336, 0.9179517455172623, 0.8473798317648323, 0.4762902334991442, 0.470053809525508, 0.4378007472767619, 0.3373892263842115, 0.36447120844173936, 0.2838763525972683] [1.0, 0.9860655544907314, 0.9853169097084781, 0.9853169097084781, 0.872

In [0]:
expr = "(^[+-]?([0-9]*[.])?[0-9]+)"
test_df = (df.filter(df.song_hotttnesss.rlike(expr)))

In [0]:
numeric_columns=['artist_familiarity',
 'artist_hotttnesss',
 'duration',
 'end_of_fade_in',
 'energy',
 'key',
 'key_confidence',
 'loudness',
 'song_hotttnesss',
 'tempo',
 'time_signature',
 'time_signature_confidence',
 'year']

from pyspark.sql.types import DoubleType

test_df2=test_df
for c in numeric_columns:
  test_df2=test_df2.withColumn(c,test_df2[c].cast(DoubleType()))

In [0]:
display(test_df2.dtypes)

_1,_2
artist_familiarity,double
artist_hotttnesss,double
artist_id,string
artist_location,string
artist_mbtags,string
artist_mbtags_count,string
artist_name,string
artist_terms,string
artist_terms_freq,string
artist_terms_weight,string


In [0]:
test_df2 = test_df2.na.drop()
test_df.drop_duplicates()

Out[9]: DataFrame[artist_familiarity: double, artist_hotttnesss: double, artist_id: string, artist_location: string, artist_mbtags: string, artist_mbtags_count: string, artist_name: string, artist_terms: string, artist_terms_freq: string, artist_terms_weight: string, danceability: string, duration: string, end_of_fade_in: string, energy: string, key: string, key_confidence: string, loudness: string, mode: string, mode_confidence: string, release: string, segments_confidence: string, segments_loudness_max: string, segments_loudness_max_time: string, segments_pitches: string, segments_timbre: string, similar_artists: string, song_hotttnesss: string, song_id: string, start_of_fade_out: string, tempo: string, time_signature: string, time_signature_confidence: string, title: string, track_id: string, year: string]

In [0]:
test_df = test_df2.select(['artist_hotttnesss', 'loudness', 'song_hotttnesss'])

In [0]:
test_df.cache()

Out[11]: DataFrame[artist_hotttnesss: double, loudness: double, song_hotttnesss: double]

In [0]:
from pyspark.mllib.regression import LinearRegressionWithSGD
from pyspark.ml.feature import VectorAssembler
from pyspark.mllib.regression import LabeledPoint
from pyspark.ml.regression import LinearRegression
from pyspark.ml import Pipeline

assembler = VectorAssembler(
    inputCols=["artist_hotttnesss", "loudness"],
    outputCol="features2")

lr = LinearRegression(labelCol='song_hotttnesss', featuresCol='features2')

pipeline = Pipeline(stages=[assembler, lr])

model = pipeline.fit(transformed)

# data = transformed.select(transformed.song_hotttnesss.alias("label"), transformed.features).rdd.map(lambda x: LabeledPoint(x.label, x.features))

# first_model = LinearRegressionWithSGD.train(data=data, 
#                                            iterations=500, 
#                                            step=0.1,
#                                            miniBatchFraction=1.0, 
#                                            initialWeights=None, 
#                                            regParam=1e-1, 
#                                            regType='l2', 
#                                            intercept=True)

Out[18]: PipelineModel_59bac93db264